In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to parse the page and return the data and the next page URL
def parse_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    store_data = soup.find_all('div', class_='item-information')
    page_data = []
    
    for store in store_data:
        # Extract the main details from the listing
        location_element = store.find(class_='building-address')
        property_element = store.find(class_='building key-value')
        availability_element = store.find(class_='availability key-value')
        price_element = store.find(class_='price key-value')

        Location = location_element.text.strip() if location_element else "Not found"
        Property = property_element.text.strip() if property_element else "Not found"
        Availability = availability_element.text.strip() if availability_element else "Not found"
        Price = price_element.text.strip() if price_element else "Not found"

        # Extract the 'View Details' URL
        details_link = store.find('a', class_='btn btn-ghost-primary')
        details_url = details_link['href'] if details_link else None

        # Extract additional details from the detail page
        if details_url:
            detail_page_data = parse_details_page(details_url)
        else:
            detail_page_data = {}

        # Combine data and append to the page_data list
        page_data.append({
            'building-address': Location, 
            'building key-value': Property,
            'availability key-value': Availability,
            'price key-value': Price,
            **detail_page_data  # Merge dictionaries
        })

    # Find the link to the next page
    next_page_element = soup.find('a', attrs={'class': 'btn btn-default btn-sm', 'aria-label': 'Next Page'})
    next_page_url = next_page_element['href'] if next_page_element and next_page_element.get('href') else None
    
    return page_data, next_page_url

# Function to parse the details page and return additional data
def parse_details_page(url):
    response = requests.get(url)
    details_soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the walkscore-section list
    walkscore_section = details_soup.find('ul', class_='walkscore-section')
    scores_data = {}
    
    if walkscore_section:
        # Find all list items within the walkscore-section
        score_items = walkscore_section.find_all('li')
        
        # Iterate over each score item
        for item in score_items:
            score_type_element = item.find('p')
            if score_type_element:
                score_type = score_type_element.get_text(strip=True).split(' ')[0]  # Split to get the type of score e.g., "Walk", "Transit"
                score_value = item.find('span').get_text(strip=True)
                score_description = item.find('span', class_='ws-description').get_text(strip=True)
                
                scores_data[f'{score_type} Score Value'] = score_value
                scores_data[f'{score_type} Score Info'] = score_description
                
    else:
        # If the section is not found, set default messages
        scores_data = {
            'Walk Score Info': "Not available",
            'Walk Score Value': "Not available",
            'Transit Score Info': "Not available",
            'Transit Score Value': "Not available",
            'Bike Score Info': "Not available",
            'Bike Score Value': "Not available",
        }
    
    return scores_data

    

        
# Initialize the list to hold all data
all_data = []

# Start with the initial URL
base_url = 'https://www.commercialcafe.com/retail/ca/On/'
next_page_url = base_url

# Loop through pages and collect data
while next_page_url:
    print(f'Scraping {next_page_url}')  # Print the URL being scraped for reference
    page_data, next_page_url = parse_page(next_page_url)
    all_data.extend(page_data)
    
    # If 'next_page_url' is relative, you may need to join it with the base URL
    if next_page_url and not next_page_url.startswith('http'):
        next_page_url = requests.compat.urljoin(base_url, next_page_url)
    
    # Polite delay to not overwhelm the server
    time.sleep(1)

# Convert the list to a DataFrame and save to Excel
df = pd.DataFrame(all_data)
df.to_excel('extracted_data.xlsx', index=False)

Scraping https://www.commercialcafe.com/retail/ca/ON/?
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=2
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=3
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=4
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=5
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=6
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=7
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=8
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=9
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=10
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=11
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=12
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=13
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=14
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=15
Scraping https://www.commercialcafe.com/retail/ca/on/?Page=16
Scraping https://www.co

In [274]:
# Convert the list to a DataFrame and save to Excel
df = pd.DataFrame(all_data)

In [275]:


# Define a function to split the address field based on the observed pattern
def split_address(address):
    # Check if address is NaN or None
    if pd.isnull(address):
        return [None] * 4

    # Initialize the parts list with None
    parts = [None] * 4
    
    # Split the address using comma as the separator
    components = [comp.strip() for comp in address.split(',')]
    
    # Assign the split components to the respective parts based on their positions
    if len(components) == 4:
        # All parts are present
        parts = components
    elif len(components) == 3:
        # Neighborhood might be missing, which is the second component
        parts[0] = components[0]  # Street Address
        parts[2] = components[1]  # City
        parts[3] = components[2]  # Province
    elif len(components) == 2:
        # Only Street Address and City are present
        parts[0] = components[0]  # Street Address
        parts[2] = components[1]  # City

    
    return parts

# Apply the function to each row in the 'building-address' column
address_parts = df['building-address'].apply(split_address)

# Create new columns for each part of the address
df[['Street Address', 'Neighborhood', 'City', 'Province']] = pd.DataFrame(address_parts.tolist(), index=df.index)
# Drop the original 'building-address' column from the DataFrame
df.drop('building-address', axis=1, inplace=True)

# Move the new address columns to the front
cols_to_front = ['Street Address', 'Neighborhood', 'City', 'Province']
df = df[cols_to_front + [col for col in df.columns if col not in cols_to_front]]

# Display the DataFrame to confirm the columns have been re-ordered
df.head()








,Street Address,Neighborhood,City,Province,building key-value,availability key-value,price key-value,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info
0,146 Yonge Street,Financial District,Toronto,ON,"Property\n\nMixed Use\n12,000 SF","Availability\n\n2 Spaces\n8,800 SF",For Lease\n\nContact for pricing,100/100,Walker's Paradise,100/100,Rider's Paradise,91/100,Biker's Paradise
1,1324 Windsor Avenue,The Core,Windsor,ON,"Property\n\nRetail\n25,000 SF","Availability\n\n3 Spaces\n37,250 SF",For Lease\n\n$7.20 - $12.00/SF/YR,78/100,Very Walkable,48/100,Some Transit,60/100,Bikeable
2,2194 Robertson Road,Bells Corners,Ottawa,ON,"Property\n\nRetail\n43,939 SF","Availability\n\n1 Space\n4,454 SF",For Lease\n\n$20.00/SF/YR,71/100,Very Walkable,44/100,Some Transit,68/100,Bikeable
3,700 Strasburg Road,Country Hills West,Kitchener,ON,"Property\n\nRetail\n2,065 SF","Availability\n\n1 Space\n2,350 SF",For Lease\n\n$21.00/SF/YR,71/100,Very Walkable,51/100,Good Transit,72/100,Very Bikeable
4,1585 Victoria Street N,Grand River North,Kitchener,ON,"Property\n\nRetail\n9,718 SF","Availability\n\n2 Spaces\n14,577 SF",For Lease\n\n$12.00 - $13.50/SF/YR,36/100,Car-Dependent,34/100,Some Transit,50/100,Bikeable


In [276]:


# Split the 'building key-value' column into 'PropertyType' and 'AreaValue' columns
df['PropertyType'] = df['building key-value'].str.extract(r'(Retail|Mixed Use)')
df['Total Area'] = df['building key-value'].str.extract(r'(\d+,\d+|\d+) SF')

# Split the 'availability key-value' column into 'Number of spaces Available' and 'Area per space' columns
df['Number of Spaces Available'] = df['availability key-value'].str.extract(r'(\d+)\s*Space')
df['Area per space'] = df['availability key-value'].str.extract(r'(\d+,*\d+)\s*SF')

df['Number of Spaces Available'] = df['Number of Spaces Available'].fillna(0)
df['Area per space'] = df['Area per space'].fillna('Na')

# Remove the 'Property' text from the 'Area' column
df['building key-value'] = df['building key-value'].str.replace('Property', '')
# Remove the 'Availability' text from the 'availability key-value' column
df['availability key-value'] = df['availability key-value'].str.replace("Availability","")
# Remove the 'Availability' text from the 'availability key-value' column
df['price key-value'] = df['price key-value'].str.replace("For Lease","")

df = df.drop('building key-value', axis=1)
df = df.drop('availability key-value', axis=1)

# Let's first fix the 'price key-value' column by replacing newline characters with a space and renaming it to 'Price'

df['Price'] = df['price key-value'].replace({r'\n': ' '}, regex=True)

# Now we can drop the original 'price key-value' column as we have a clean 'Price' column
df.drop('price key-value', axis=1, inplace=True)

# The columns to move after 'Province' based on the image provided
cols_to_move = ['PropertyType', 'Total Area', 'Number of Spaces Available', 'Area per space', 'Price', 
                'Walk Score Value', 'Walk Score Info', 'Transit Score Value', 'Transit Score Info', 
                'Bike Score Value', 'Bike Score Info']

# Let's identify all the other columns that will remain in their original position
other_cols = [col for col in df.columns if col not in cols_to_move and col not in ['Street Address','Neighborhood','City','Province']]

# New column order with the specified columns moved right after 'Province'
new_order = ['Street Address','Neighborhood','City','Province'] + cols_to_move + other_cols

# Reorder the columns in the DataFrame
df = df[new_order]

# Display the DataFrame to confirm the rearrangement
df.head()






,Street Address,Neighborhood,City,Province,PropertyType,Total Area,Number of Spaces Available,Area per space,Price,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info
0,146 Yonge Street,Financial District,Toronto,ON,Mixed Use,"12,000",2,"8,800",Contact for pricing,100/100,Walker's Paradise,100/100,Rider's Paradise,91/100,Biker's Paradise
1,1324 Windsor Avenue,The Core,Windsor,ON,Retail,"25,000",3,"37,250",$7.20 - $12.00/SF/YR,78/100,Very Walkable,48/100,Some Transit,60/100,Bikeable
2,2194 Robertson Road,Bells Corners,Ottawa,ON,Retail,"43,939",1,"4,454",$20.00/SF/YR,71/100,Very Walkable,44/100,Some Transit,68/100,Bikeable
3,700 Strasburg Road,Country Hills West,Kitchener,ON,Retail,"2,065",1,"2,350",$21.00/SF/YR,71/100,Very Walkable,51/100,Good Transit,72/100,Very Bikeable
4,1585 Victoria Street N,Grand River North,Kitchener,ON,Retail,"9,718",2,"14,577",$12.00 - $13.50/SF/YR,36/100,Car-Dependent,34/100,Some Transit,50/100,Bikeable


In [277]:
df.head()

,Street Address,Neighborhood,City,Province,PropertyType,Total Area,Number of Spaces Available,Area per space,Price,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info
0,146 Yonge Street,Financial District,Toronto,ON,Mixed Use,"12,000",2,"8,800",Contact for pricing,100/100,Walker's Paradise,100/100,Rider's Paradise,91/100,Biker's Paradise
1,1324 Windsor Avenue,The Core,Windsor,ON,Retail,"25,000",3,"37,250",$7.20 - $12.00/SF/YR,78/100,Very Walkable,48/100,Some Transit,60/100,Bikeable
2,2194 Robertson Road,Bells Corners,Ottawa,ON,Retail,"43,939",1,"4,454",$20.00/SF/YR,71/100,Very Walkable,44/100,Some Transit,68/100,Bikeable
3,700 Strasburg Road,Country Hills West,Kitchener,ON,Retail,"2,065",1,"2,350",$21.00/SF/YR,71/100,Very Walkable,51/100,Good Transit,72/100,Very Bikeable
4,1585 Victoria Street N,Grand River North,Kitchener,ON,Retail,"9,718",2,"14,577",$12.00 - $13.50/SF/YR,36/100,Car-Dependent,34/100,Some Transit,50/100,Bikeable


In [278]:
df['Walk Score Value'] = df['Walk Score Value'].astype(str)
df['Walk Score Value'] = df['Walk Score Value'].str.extract('(\d+)/100').astype(float)
df['Transit Score Value'] = df['Transit Score Value'].astype(str)
df['Transit Score Value'] = df['Transit Score Value'].str.extract('(\d+)/100').astype(float)
df['Bike Score Value'] = df['Bike Score Value'].astype(str)
df['Bike Score Value'] = df['Bike Score Value'].str.extract('(\d+)/100').astype(float)

print(df)


             Street Address        Neighborhood       City Province  \
0          146 Yonge Street  Financial District    Toronto       ON   
1       1324 Windsor Avenue            The Core    Windsor       ON   
2       2194 Robertson Road       Bells Corners     Ottawa       ON   
3        700 Strasburg Road  Country Hills West  Kitchener       ON   
4    1585 Victoria Street N   Grand River North  Kitchener       ON   
..                      ...                 ...        ...      ...   
515  34 Highbury Park Drive           Barrhaven     Ottawa       ON   
516  1318 Queen Street West        Roncesvalles    Toronto       ON   
517   731 Queen Street East           Riverdale    Toronto       ON   
518       560 Rideau Street          Sandy Hill     Ottawa       ON   
519    600 King Street West    Fashion District    Toronto       ON   

    PropertyType Total Area Number of Spaces Available Area per space  \
0      Mixed Use     12,000                          2          8,800   
1

In [279]:
df.columns

Index(['Street Address', 'Neighborhood', 'City', 'Province', 'PropertyType',
       'Total Area', 'Number of Spaces Available', 'Area per space', 'Price',
       'Walk Score Value', 'Walk Score Info', 'Transit Score Value',
       'Transit Score Info', 'Bike Score Value', 'Bike Score Info'],
      dtype='object')

In [280]:
# Adjust function to handle commas in the price string
def convert_price_to_float_v2(price_str):
    # Remove commas from the price string for correct float conversion
    price_str = price_str.replace(',', '')
    if 'Contact for pricing' in price_str:
        return np.nan  # Replace "Contact for pricing" with NaN
    elif '-' in price_str:
        # Calculate average if there's a range
        prices = [float(p.strip().replace('$', '').split('/')[0]) for p in price_str.split('-')]
        return sum(prices) / len(prices)
    else:
        # Extract and return the float value directly
        return float(price_str.replace('$', '').split('/')[0])

# Apply the adjusted conversion function to the Price column
df['Price (/SF/YR)'] = df['Price'].apply(convert_price_to_float_v2)

# Drop the original Price column
df.drop(columns=['Price'], inplace=True)

# Display the first few rows of the dataframe to verify the changes
df.head()


,Street Address,Neighborhood,City,Province,PropertyType,Total Area,Number of Spaces Available,Area per space,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info,Price (/SF/YR)
0,146 Yonge Street,Financial District,Toronto,ON,Mixed Use,"12,000",2,"8,800",100.0,Walker's Paradise,100.0,Rider's Paradise,91.0,Biker's Paradise,NaN
1,1324 Windsor Avenue,The Core,Windsor,ON,Retail,"25,000",3,"37,250",78.0,Very Walkable,48.0,Some Transit,60.0,Bikeable,9.60
2,2194 Robertson Road,Bells Corners,Ottawa,ON,Retail,"43,939",1,"4,454",71.0,Very Walkable,44.0,Some Transit,68.0,Bikeable,20.00
3,700 Strasburg Road,Country Hills West,Kitchener,ON,Retail,"2,065",1,"2,350",71.0,Very Walkable,51.0,Good Transit,72.0,Very Bikeable,21.00
4,1585 Victoria Street N,Grand River North,Kitchener,ON,Retail,"9,718",2,"14,577",36.0,Car-Dependent,34.0,Some Transit,50.0,Bikeable,12.75


In [281]:
# Adjust function to use the correct column name "Area per space"
def adjust_price_v2(row):
    if row['Price (/SF/YR)'] >= 50:  # This checks if the value is likely a monthly rent
        # Convert Area per space to float and handle commas
        try:
            area_per_space = float(row['Area per space'].replace(',', ''))
        except ValueError:  # In case the conversion fails, it might be due to invalid formatting
            return row['Price (/SF/YR)']  # Return the original price without adjustment
        # Adjust the price from /MO to /SF/YR
        adjusted_price = (row['Price (/SF/YR)'] * 12) / area_per_space
        return adjusted_price
    else:
        return row['Price (/SF/YR)']

# Apply the adjusted function to each row
df['Price (/SF/YR)'] = df.apply(adjust_price_v2, axis=1)

# Display the first few rows to verify the changes
df.head()


,Street Address,Neighborhood,City,Province,PropertyType,Total Area,Number of Spaces Available,Area per space,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info,Price (/SF/YR)
0,146 Yonge Street,Financial District,Toronto,ON,Mixed Use,"12,000",2,"8,800",100.0,Walker's Paradise,100.0,Rider's Paradise,91.0,Biker's Paradise,NaN
1,1324 Windsor Avenue,The Core,Windsor,ON,Retail,"25,000",3,"37,250",78.0,Very Walkable,48.0,Some Transit,60.0,Bikeable,9.60
2,2194 Robertson Road,Bells Corners,Ottawa,ON,Retail,"43,939",1,"4,454",71.0,Very Walkable,44.0,Some Transit,68.0,Bikeable,20.00
3,700 Strasburg Road,Country Hills West,Kitchener,ON,Retail,"2,065",1,"2,350",71.0,Very Walkable,51.0,Good Transit,72.0,Very Bikeable,21.00
4,1585 Victoria Street N,Grand River North,Kitchener,ON,Retail,"9,718",2,"14,577",36.0,Car-Dependent,34.0,Some Transit,50.0,Bikeable,12.75


In [282]:
# Clean the numeric data by removing commas and converting the values to floats
numeric_columns = ['Total Area', 'Number of Spaces Available', 'Area per space', 'Walk Score Value', 'Transit Score Value', 'Bike Score Value']

# Remove commas and convert columns to numeric
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[numeric_columns])

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Impute the missing values in the 'Price' column
X_imputed = imputer.fit_transform(X_scaled)

# Since we don't have a separate target variable for 'Price', we will assume that it's another column in the dataset that needs imputation.
# Normally, you would use the imputed features to predict 'Price' with a KNN regressor, but here we'll just demonstrate the imputation.
# Assign the imputed and rescaled values back to the DataFrame
df[numeric_columns] = scaler.inverse_transform(X_imputed)

# Check the first few rows to ensure the operation was successful
df.head()




,Street Address,Neighborhood,City,Province,PropertyType,Total Area,Number of Spaces Available,Area per space,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info,Price (/SF/YR)
0,146 Yonge Street,Financial District,Toronto,ON,Mixed Use,12000.0,2.0,8800.0,100.0,Walker's Paradise,100.0,Rider's Paradise,91.0,Biker's Paradise,NaN
1,1324 Windsor Avenue,The Core,Windsor,ON,Retail,25000.0,3.0,37250.0,78.0,Very Walkable,48.0,Some Transit,60.0,Bikeable,9.60
2,2194 Robertson Road,Bells Corners,Ottawa,ON,Retail,43939.0,1.0,4454.0,71.0,Very Walkable,44.0,Some Transit,68.0,Bikeable,20.00
3,700 Strasburg Road,Country Hills West,Kitchener,ON,Retail,2065.0,1.0,2350.0,71.0,Very Walkable,51.0,Good Transit,72.0,Very Bikeable,21.00
4,1585 Victoria Street N,Grand River North,Kitchener,ON,Retail,9718.0,2.0,14577.0,36.0,Car-Dependent,34.0,Some Transit,50.0,Bikeable,12.75


In [283]:
# Function to adjust the values as specified
def adjust_decimal(value):
    # Check if the value is less than 1 and not NaN
    if pd.notnull(value) and value < 1:
        # Shift the decimal point two spaces to the right
        value *= 100
    # Round the value to two decimal places
    return round(value, 2)

# Apply the function to the 'Price (/SF/YR)' column
df['Price (/SF/YR)'] = df['Price (/SF/YR)'].apply(adjust_decimal)

# Display the first few rows to verify the changes
df.head()


,Street Address,Neighborhood,City,Province,PropertyType,Total Area,Number of Spaces Available,Area per space,Walk Score Value,Walk Score Info,Transit Score Value,Transit Score Info,Bike Score Value,Bike Score Info,Price (/SF/YR)
0,146 Yonge Street,Financial District,Toronto,ON,Mixed Use,12000.0,2.0,8800.0,100.0,Walker's Paradise,100.0,Rider's Paradise,91.0,Biker's Paradise,NaN
1,1324 Windsor Avenue,The Core,Windsor,ON,Retail,25000.0,3.0,37250.0,78.0,Very Walkable,48.0,Some Transit,60.0,Bikeable,9.60
2,2194 Robertson Road,Bells Corners,Ottawa,ON,Retail,43939.0,1.0,4454.0,71.0,Very Walkable,44.0,Some Transit,68.0,Bikeable,20.00
3,700 Strasburg Road,Country Hills West,Kitchener,ON,Retail,2065.0,1.0,2350.0,71.0,Very Walkable,51.0,Good Transit,72.0,Very Bikeable,21.00
4,1585 Victoria Street N,Grand River North,Kitchener,ON,Retail,9718.0,2.0,14577.0,36.0,Car-Dependent,34.0,Some Transit,50.0,Bikeable,12.75


In [287]:
from sklearn.neighbors import KNeighborsRegressor

# Since we've already converted non-numeric 'Price' values to NaN, we will split the data into known and unknown price datasets
known_price_df = df.dropna(subset=['Price (/SF/YR)'])
unknown_price_df = df[df['Price (/SF/YR)'].isna()]

# Define the feature matrix (X) and target vector (y)
X_known = known_price_df[numeric_columns]
y_known = known_price_df['Price (/SF/YR)']

# Fit the scaler on known price data and transform both known and unknown sets
scaler = StandardScaler()
X_known_scaled = scaler.fit_transform(X_known)
X_unknown_scaled = scaler.transform(unknown_price_df[numeric_columns])

# Initialize the KNN regressor model
knn_regressor = KNeighborsRegressor(n_neighbors=5)

# Fit the model on known price data
knn_regressor.fit(X_known_scaled, y_known)

# Predict the prices for the rows with unknown prices
unknown_price_df['Price (/SF/YR)'] = knn_regressor.predict(X_unknown_scaled)

# Combine the datasets back together
df_combined = pd.concat([known_price_df, unknown_price_df], ignore_index=True)

# Display the DataFrame with imputed prices
df_combined[['Street Address', 'Neighborhood', 'City', 'Province', 'Price (/SF/YR)']].head()
df_combined['Price (/SF/YR)'] =df_combined['Price (/SF/YR)'].round(decimals=2)

C:\Users\user\AppData\Local\Temp\ipykernel_24856\3032766648.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_price_df['Price (/SF/YR)'] = knn_regressor.predict(X_unknown_scaled)


In [288]:


df_combined.to_excel('Final_data.xlsx', index=False)